<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/03_embeddings_sherlock_holmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings de Palabras a partir de libros de Sherlock Holmes




# 0) Preparación de librerías


In [ ]:
!pip install numpy==1.26.4 scipy==1.13.1 gensim==4.3.3 spacy==3.7.5

# 1) Carga de Datos

Carga libros publicados en Project Gutenberg y elimina el header/footer:

In [1]:
import urllib.request
from pathlib import Path

def load_gutenberg_text(url):
    print("Cargando texto desde:", url, "..")
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8')

    # Encontrar el inicio y fin del contenido real (eliminar header/footer de Gutenberg)
    start = raw.find("*** START OF THE PROJECT GUTENBERG")
    start = raw.find("\n", start) + 1
    end = raw.find("*** END OF THE PROJECT GUTENBERG")

    return raw[start:end]

# URLs de los libros en Project Gutenberg
urls = [
    "https://www.gutenberg.org/files/1661/1661-0.txt",    # The Adventures of Sherlock Holmes
    "https://www.gutenberg.org/files/108/108-0.txt",      # The Return of Sherlock Holmes
    "https://www.gutenberg.org/files/2097/2097-0.txt",    # The Hound of the Baskervilles
    "https://www.gutenberg.org/files/244/244-0.txt"       # A Study in Scarlet
]

# Carga y guarda el texto
text = " "
for url in urls:
  book_text = load_gutenberg_text(url)
  text += book_text


print("Texto cargado:", len(text), "caracteres")

Cargando texto desde: https://www.gutenberg.org/files/1661/1661-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/108/108-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/2097/2097-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/244/244-0.txt ..
Texto cargado: 1742150 caracteres


# 2) Preprocesamiento

Tokeniza y limpia el texto:
* Convierte a minúsculas
* Elimina tokens que no son palabras

In [2]:
import spacy
from spacy.lang.en import English

# Inicializar spaCy (solo tokenizador para velocidad)
nlp = English(disable=['tagger','parser','ner'])

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc
            if token.text.strip() and not token.is_punct]

# Dividir en oraciones y tokenizar
corpus_sentences = []
for line in text.split('\n'):
    if line.strip():  # ignorar líneas vacías
        tokens = tokenize(line)
        if tokens:  # ignorar líneas sin tokens válidos
            corpus_sentences.append(tokens)

print("Total de oraciones procesadas:", len(corpus_sentences))

Total de oraciones procesadas: 27683


# 3) Modelo Word2Vec

---



##3.1) Entrenamiento

In [4]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences=corpus_sentences,
                    vector_size=300,      # Dimensión del vector (300)
                    window=8,             # Ventana más amplia para capturar más contexto (8)
                    min_count=2,          # Filtrar palabras poco frecuentes (5)
                    workers=4,
                    sg=1,                 # Usar Skip-gram
                    epochs= 30,           # Más ciclos de entrenamiento
                    negative= 15,         # Tamaño de Muestra Negativa (Negative sampling)
                    alpha= 0.025,         # Learning rate inicial
                    min_alpha= 0.0001     # Learning rate final
                  )

Guardar modelos

In [5]:
w2v_model.save("sherlock_w2v.model")

## 3.2) Análisis de Similitudes

Cargar vectores

In [6]:
w2v_vectors = w2v_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", w2v_vectors.vector_size)
print("Número de palabras (Word2Vec):", len(w2v_vectors.index_to_key))


Estadísticas del modelo:
Dimensión de los vectores: 300
Número de palabras (Word2Vec): 8416


In [7]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('sherlock', 0.5204015374183655), ('demurely', 0.4592164158821106), ('mr', 0.4341519773006439), ('cheerily', 0.43188294768333435), ('approvingly', 0.4205712080001831), ('gleefully', 0.4183337986469269), ('involuntarily', 0.41294097900390625), ('bungler', 0.40235358476638794), ('compliment', 0.40162819623947144), ('motioning', 0.4008224606513977)]


In [8]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(w2v_vectors.most_similar('crime'))



Palabras más similares a 'crime' (Word2Vec):
[('committed', 0.5234676599502563), ('literature', 0.47155189514160156), ('deliberate', 0.46535801887512207), ('featureless', 0.4565415382385254), ('logic', 0.442795991897583), ('records', 0.4420148432254791), ('perpetrator', 0.4353950023651123), ('talent', 0.43511196970939636), ('insane', 0.4232688248157501), ('detect', 0.42240411043167114)]


In [9]:
print("\nSimilitud entre 'crime' y 'art':")
print("Word2Vec:", w2v_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
Word2Vec: 0.2433517


## 3.3) Palabras fuera de vocabulario

In [ ]:
print("\nPrueba con palabra fuera de vocabulario:")
try:
    print("Word2Vec - Similares a 'investigador':")
    print(w2v_vectors.most_similar('investigador'))
except KeyError:
    print("Word2Vec no puede manejar palabras fuera de vocabulario")

## 3.4) Analogías

In [ ]:
print("\nAnalogías (Word2Vec):")
result = w2v_vectors.most_similar(positive=['watson', 'crime'],
                                negative=['holmes'])
print("watson:holmes como crime:?")
print(result)

#4) Modelo FastText


##4.1) Entrenamiento

In [ ]:
from gensim.models import FastText
ft_model = FastText(sentences=corpus_sentences,
                   vector_size=300,    # Aumentar dimensionalidad (300)
                   window=8,           # Ventana más amplia para capturar más contexto (8)
                   min_count=2,        # Mantener min_count bajo para capturar más variantes
                   workers=4,
                   sg=1,               # Skip-gram para mejor calidad
                   min_n=2,            # Tamaño mínimo de n-gramas
                   max_n=6,            # Tamaño máximo de n-gramas (aumentado para capturar más patrones)
                   epochs=30,          # Más ciclos de entrenamiento (30)
                   word_ngrams=1,      # Habilitar n-gramas de palabras
                   negative=15,        # Más muestras negativas
                   alpha=0.025,        # Learning rate inicial
                   min_alpha=0.0001    # Learning rate final
              )

Almacenamiento del modelo:

In [ ]:
ft_model.save("sherlock_ft.model")

## 4.2) Análisis de Similitudes

In [ ]:
ft_vectors = ft_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", ft_vectors.vector_size)
print("Número de palabras (FastText):", len(ft_vectors.index_to_key))

In [ ]:
print("\nPalabras más similares a 'holmes' (FastText):")
print(ft_vectors.most_similar('holmes'))

In [ ]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(ft_vectors.most_similar('crime'))

In [ ]:
print("\nSimilitud entre 'crime' y 'art':")
print("FastText:", ft_vectors.similarity('crime', 'art'))

##4.3) Palabras fuera del vocabulario

In [ ]:
print("\nPrueba con palabra fuera de vocabulario:")

print("\nFastText - Similares a 'investigador':")
print(ft_vectors.most_similar('investigador'))  # FastText puede generar vectores para palabras nuevas
